Parallel Processing in Python
====

You will almost always start from the question, "How can I break up this problem into smaller pieces that can run concurrently?"

Once you have an answer to that question, there are a few Python tools that can help implement that answer.

Multiprocessing
----

In [1]:
import multiprocessing
import time
import numpy.random as rand

data = [(a,rand.uniform(0,1)) for a in 'abcdefghijklmnop']

def mp_worker(data):
    inputs, the_time = data
    print(" Processs %s\tWaiting %s seconds" % (inputs, the_time))
    time.sleep(float(the_time))
    print(" Process %s\tDONE" % inputs)
    return inputs.upper()

def mp_handler(N, workerfn):
    p = multiprocessing.Pool(N)
    return p.map(workerfn, data)

In [2]:
[ mp_worker(d) for d in data ]

 Processs a	Waiting 0.22864302472473164 seconds
 Process a	DONE
 Processs b	Waiting 0.7800559317449411 seconds
 Process b	DONE
 Processs c	Waiting 0.9161706026859643 seconds
 Process c	DONE
 Processs d	Waiting 0.08122714846798784 seconds
 Process d	DONE
 Processs e	Waiting 0.31696670010750305 seconds
 Process e	DONE
 Processs f	Waiting 0.8325693924512009 seconds
 Process f	DONE
 Processs g	Waiting 0.9781958475357412 seconds
 Process g	DONE
 Processs h	Waiting 0.03588696529716129 seconds
 Process h	DONE
 Processs i	Waiting 0.7868422379478608 seconds
 Process i	DONE
 Processs j	Waiting 0.10519346133768359 seconds
 Process j	DONE
 Processs k	Waiting 0.10052092766703347 seconds
 Process k	DONE
 Processs l	Waiting 0.95410673291936 seconds
 Process l	DONE
 Processs m	Waiting 0.6651722931394547 seconds
 Process m	DONE
 Processs n	Waiting 0.27178399452351487 seconds
 Process n	DONE
 Processs o	Waiting 0.09043737956778442 seconds
 Process o	DONE
 Processs p	Waiting 0.10002843157232932 seconds
 

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P']

In [3]:
mp_handler(3, mp_worker)

 Processs a	Waiting 0.22864302472473164 seconds
 Processs c	Waiting 0.9161706026859643 seconds
 Processs e	Waiting 0.31696670010750305 seconds
 Process a	DONE
 Processs b	Waiting 0.7800559317449411 seconds
 Process e	DONE
 Processs f	Waiting 0.8325693924512009 seconds
 Process c	DONE
 Processs d	Waiting 0.08122714846798784 seconds
 Process d	DONE
 Processs g	Waiting 0.9781958475357412 seconds
 Process b	DONE
 Processs i	Waiting 0.7868422379478608 seconds
 Process f	DONE
 Processs k	Waiting 0.10052092766703347 seconds
 Process k	DONE
 Processs l	Waiting 0.95410673291936 seconds
 Process i	DONE
 Processs j	Waiting 0.10519346133768359 seconds
 Process j	DONE
 Processs m	Waiting 0.6651722931394547 seconds
 Process g	DONE
 Processs h	Waiting 0.03588696529716129 seconds
 Process h	DONE
 Processs o	Waiting 0.09043737956778442 seconds
 Process o	DONE
 Processs p	Waiting 0.10002843157232932 seconds
 Process l	DONE
 Process p	DONE
 Process m	DONE
 Processs n	Waiting 0.27178399452351487 seconds
 

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P']

In [4]:
import numpy as np

def bigpower(power):
    d = np.random.randn(100000000)**power
    print("Raising random array to the {0}th power".format(power))
    return d.mean()

data = [1, 2, 12, 15]

In [5]:
mp_handler(3, bigpower)

Raising random array to the 1th power
Raising random array to the 2th power
Raising random array to the 12th power
Raising random array to the 15th power


[0.0001303914200917801,
 1.0001200466724247,
 10491.170994989114,
 -11207.290118756477]

Threading
----

Threads are lighter-weight since they share the Python interpreter and can sometimes share data. But mind the GIL!

In [6]:
import threading
import queue

In [7]:
q = queue.Queue()

In [8]:
q.put('foo')

In [9]:
q.put(5)

In [10]:
q.put('even more')

In [14]:
q.get(block=False)

Empty: 

In [15]:
def work():
    q.put(np.random.randn(1000))

In [16]:
t = threading.Thread(target=work)

In [17]:
t

<Thread(Thread-10, initial)>

In [18]:
t.start()

In [19]:
q.get(block=False)

array([-4.92693949e-01,  4.33164572e-01, -6.57958960e-01, -6.55518893e-01,
        4.80213790e-01,  2.76569601e-01,  5.38388927e-01,  6.20120985e-01,
        1.57126069e+00, -1.54073523e+00,  7.21808979e-01,  1.11779226e+00,
       -8.66582927e-01, -5.73608634e-01,  1.98455488e+00, -1.30791146e+00,
       -5.02746732e-01,  1.04661582e-01,  5.36953165e-01,  6.74301239e-01,
        4.60492387e-01, -3.59980961e-01, -8.30594506e-01,  1.27553604e+00,
       -3.57542930e-01,  6.96072078e-01, -2.83956750e-01, -7.36619180e-01,
       -2.36631143e-01,  1.26154559e+00,  9.42282828e-01,  1.07661620e+00,
        2.01835486e+00, -1.02002578e+00, -2.02023175e+00,  1.93775153e-01,
        2.19388456e+00, -1.88266033e+00,  8.02695560e-01, -2.36610605e+00,
       -2.40218910e+00,  1.03778558e-01,  9.26869600e-01, -3.58257251e-01,
        4.30790507e-01,  1.67906131e-01,  2.86715177e-01,  6.74522614e-01,
        2.03836205e-01, -9.30721640e-01, -2.02925597e+00, -1.33973835e+00,
       -1.87178810e+00,  

Dask
---

Higher level abstractions are available!

In [20]:
import numpy as np
import dask.array as da
import memory_profiler

In [21]:
Y = da.random.normal(size=(1000, 1000),
                     chunks=(100, 100))

Y

dask.array<normal, shape=(1000, 1000), dtype=float64, chunksize=(100, 100), chunktype=numpy.ndarray>

In [22]:
mu = Y.mean(axis=0)
mu

dask.array<mean_agg-aggregate, shape=(1000,), dtype=float64, chunksize=(100,), chunktype=numpy.ndarray>

In [23]:
mu.sum()

dask.array<sum-aggregate, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

Notice the computation hasn't actually happened yet...

In [24]:
mu[0].compute()

0.03043790253633331

In [25]:
from dask.diagnostics import ProgressBar

with ProgressBar():
    mu = Y.mean().sum().compute()

[########################################] | 100% Completed |  0.3s


In [26]:
mu

-9.942397897661555e-05